<H1>Sentiment Analyse Deutsch</H1>

In [17]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import datetime
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import emoji
import requests

In [2]:
df = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Final_DE/Dataset_Wirecard.csv", sep=";",
                 parse_dates=["created_at"])

<h2>BERT Sentiment Analyse<h2>

In [3]:
#remove duplicate emojis
def remove_duplicate_emoji(orig_str):
    prev_emoji = None
    remove_duplicate_emoji = []
    for c in orig_str:
        if c in emoji.EMOJI_DATA:
            if prev_emoji == c:
                continue
            prev_emoji = c
        remove_duplicate_emoji.append(c)
    return "".join(remove_duplicate_emoji)

#remove all emojis
def de_emojify(text):
    regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
                           "]+", flags = re.UNICODE)
    return regex_pattern.sub(r'', text)

In [4]:
model_name = "oliverguhr/german-sentiment-bert"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [6]:
classifier(df.iloc[39]["text"])

[{'label': 'negative', 'score': 0.9396710395812988}]

In [9]:
emoji.demojize(df.iloc[39]["text"], language="de")

' Freue mich drauf :süffisant_lächelndes_gesicht:'

In [10]:
classifier(emoji.demojize(df.iloc[39]["text"], language="de"))

[{'label': 'positive', 'score': 0.9853807091712952}]

In [ ]:
#Zum Testen
for each in range(37400,37405):
    print(each)
    try:
        if len(emoji.demojize(df.iloc[each]["text"])) >= 425:
            print(each, " in if 1")
            if len(emoji.demojize(remove_duplicate_emoji(df.iloc[each]["text"]))) >= 425:
                print(each, " in if 2")
                if len(de_emojify(df.iloc[each]["text"])) >= 425:
                    print(each, " in if 3")
                    continue
                else:
                    print("Add ohne Emojis")
                    print(classifier(de_emojify(df.iloc[each]["text"]))[0]["label"])
            else:
                print("Add ohne doppelte Emojis")
                print(classifier(emoji.demojize(remove_duplicate_emoji(df.iloc[each]["text"])))[0]["label"])
        else:
            print("Add")
            print(classifier(emoji.demojize(df.iloc[each]["text"]))[0]["label"])
    except Exception:
        print(each, "neutral")

In [11]:
#Produktiv
list_sentiment_bert = []
list_error_tweets = []
n = 500
for each in range(len(df)):
    if (each/n) == 1:
        print(each)
        n = n + 500
    try:
        if len(emoji.demojize(df.iloc[each]["text"], language="de")) >= 425:
            if len(emoji.demojize(remove_duplicate_emoji(df.iloc[each]["text"]), language="de")) >= 425:
                if len(de_emojify(df.iloc[each]["text"])) >= 425:
                    continue
                else:
                    list_sentiment_bert.append(classifier(de_emojify(df.iloc[each]["text"]))[0]["label"])
            else:
                list_sentiment_bert.append(classifier(emoji.demojize(remove_duplicate_emoji(df.iloc[each]["text"]), language="de"))[0]["label"])
        else:
            list_sentiment_bert.append(classifier(emoji.demojize(df.iloc[each]["text"], language="de"))[0]["label"])
    except Exception:
        print(each)
        list_sentiment_bert.append("neutral")
        list_error_tweets.append(each)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
83500
84000
84500
85000


In [13]:
print(len(df))
print(len(list_sentiment_bert))
print(len(list_error_tweets))

114444
114444
0


In [14]:
#Ergebnisse dem DF hinzufügen
df["sentiment_bert"] = list_sentiment_bert

In [15]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,sentiment_bert
0,703311366262800384,1392131804,2016-02-26 20:11:11+00:00,703311366262800384,de,0,0,2,0,Twitter Web Client,None,None,Wirecard-Chef Markus Braun pumpt irrwitzige Su...,neutral
1,702405572012974080,1053723204,2016-02-24 08:11:53+00:00,702405572012974080,de,0,0,0,0,Twitter Web Client,None,None,Jetzt günstig einsammeln und später absahnen? ...,neutral
2,700578193334411264,19811837,2016-02-19 07:10:32+00:00,700578193334411264,de,2,0,0,0,Twitter Web Client,None,None,Wirecard: Was führt der Vorstand im Schilde? M...,neutral
3,700221272840601601,19811837,2016-02-18 07:32:15+00:00,700221272840601601,de,0,0,1,0,Twitter Web Client,None,None,Wirecard-Vorstand kauft weiter massiv eigene A...,negative
4,697324084221227008,183289696,2016-02-10 07:39:51+00:00,697324084221227008,de,0,0,0,0,dlvr.it,None,None,📈 Wirecard: Serie von Aktienkäufen des Vorstan...,neutral


In [16]:
#Backup CSV erstellen
df.to_csv("Dataset_DE_Bert.csv", sep=";", index=False)

In [73]:
df_backup = df

<h2>GerVADER</h2>

In [19]:
url = "https://gervader.p.rapidapi.com/rapidapi/sentiments"

payload = {"sentence": "dieses produkt ist hervorragend, absolute empfehlung!!"}
headers = {
	"content-type": "application/json",
	"X-RapidAPI-Key": "927b345802msh8971a9fec237922p12ab3ajsn75f56b81cbe4",
	"X-RapidAPI-Host": "gervader.p.rapidapi.com"
}

response = requests.request("POST", url, json=payload, headers=headers)

print(response.text)

{"message":"You are not subscribed to this API."}


In [22]:
df_GerVADER = df[["tweet_id"]].copy()

In [24]:
df_GerVADER["label"] = ""

In [25]:
df_GerVADER.head()

,tweet_id,label
0,703311366262800384,
1,702405572012974080,
2,700578193334411264,
3,700221272840601601,
4,697324084221227008,


In [26]:
df_GerVADER["Sentence"] = df["text"]

In [27]:
df_GerVADER.head()

,tweet_id,label,Sentence
0,703311366262800384,,Wirecard-Chef Markus Braun pumpt irrwitzige Su...
1,702405572012974080,,Jetzt günstig einsammeln und später absahnen? ...
2,700578193334411264,,Wirecard: Was führt der Vorstand im Schilde? M...
3,700221272840601601,,Wirecard-Vorstand kauft weiter massiv eigene A...
4,697324084221227008,,📈 Wirecard: Serie von Aktienkäufen des Vorstan...


In [30]:
df_GerVADER = df_GerVADER.rename(columns={"tweet_id": "ID", "label": "Label"})

In [31]:
df_GerVADER.head()

,ID,Label,Sentence
0,703311366262800384,,Wirecard-Chef Markus Braun pumpt irrwitzige Su...
1,702405572012974080,,Jetzt günstig einsammeln und später absahnen? ...
2,700578193334411264,,Wirecard: Was führt der Vorstand im Schilde? M...
3,700221272840601601,,Wirecard-Vorstand kauft weiter massiv eigene A...
4,697324084221227008,,📈 Wirecard: Serie von Aktienkäufen des Vorstan...


In [33]:
df_GerVADER.to_csv("tweets.tsv", sep="\t", index=False, header=False)

In [53]:
df_GerVader_neg = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/tests/GerVADER-master/results/mode_All/31072022_115652/GERVADER__negative.tsv",
                              sep="\t",
                             names=["text", "tweet_id", "sentiment_GerVADER", "sentiment_GerVADER_detailed"])
df_GerVader_neg.head()

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed
0,Auf Einladung von Bürgermeister Markus Braun d...,NaN,-0.2023,"{'neg': 0.114, 'neu': 0.886, 'pos': 0.0, 'comp..."
1,Wo kann man alte BRN-Beiträge hören? Wirecard...,NaN,-0.2023,"{'neg': 0.101, 'neu': 0.899, 'pos': 0.0, 'comp..."
2,Markus du nervst so hart mann!,NaN,-0.5971,"{'neg': 0.55, 'neu': 0.45, 'pos': 0.0, 'compou..."
3,"Man sollte sich überlegen, einen Englischkurs...",NaN,-0.1531,"{'neg': 0.062, 'neu': 0.892, 'pos': 0.046, 'co..."
4,Wird halt nicht jeder als Elon geboren ... a...,NaN,-0.1027,"{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'comp..."


In [54]:
df_GerVader_neu = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/tests/GerVADER-master/results/mode_All/31072022_115652/GERVADER__neutral.tsv",
                              sep="\t",
                             names=["text", "tweet_id", "sentiment_GerVADER", "sentiment_GerVADER_detailed"])
df_GerVader_neu.head()

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed
0,Wirecard-Chef Markus Braun pumpt irrwitzige Su...,NaN,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,Wirecard-Vorstand kauft weiter massiv eigene A...,NaN,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,📈 Wirecard: Serie von Aktienkäufen des Vorstan...,NaN,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Ein weiterer Goldsponsor der nativeadscamp ist...,NaN,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,Für Nerds wie Wirecard AG's (WRCDF) CEO Marku...,NaN,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


In [55]:
df_GerVader_pos = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/tests/GerVADER-master/results/mode_All/31072022_115652/GERVADER_positive.tsv",
                              sep="\t",
                             names=["text", "tweet_id", "sentiment_GerVADER", "sentiment_GerVADER_detailed"])
df_GerVader_pos.head()

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed
0,Jetzt günstig einsammeln und später absahnen? ...,NaN,0.4871,"{'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'comp..."
1,Wirecard: Was führt der Vorstand im Schilde? M...,NaN,0.3182,"{'neg': 0.0, 'neu': 0.85, 'pos': 0.15, 'compou..."
2,Im Wort Bunte steckt Braun in allen Schatti...,NaN,0.4588,"{'neg': 0.0, 'neu': 0.786, 'pos': 0.214, 'comp..."
3,"Wirecard-Aktie: Vorstandschef Braun sieht ""seh...",NaN,0.5256,"{'neg': 0.0, 'neu': 0.78, 'pos': 0.22, 'compou..."
4,Damn! Alle Hoffnungen ruhen jetzt auf Dr. Mar...,NaN,0.6467,"{'neg': 0.0, 'neu': 0.778, 'pos': 0.222, 'comp..."


In [56]:
df_GerVader = pd.concat([df_GerVader_neg, df_GerVader_neu, df_GerVader_pos],ignore_index=True)

In [57]:
df_GerVader.head()

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed
0,Auf Einladung von Bürgermeister Markus Braun d...,NaN,-0.2023,"{'neg': 0.114, 'neu': 0.886, 'pos': 0.0, 'comp..."
1,Wo kann man alte BRN-Beiträge hören? Wirecard...,NaN,-0.2023,"{'neg': 0.101, 'neu': 0.899, 'pos': 0.0, 'comp..."
2,Markus du nervst so hart mann!,NaN,-0.5971,"{'neg': 0.55, 'neu': 0.45, 'pos': 0.0, 'compou..."
3,"Man sollte sich überlegen, einen Englischkurs...",NaN,-0.1531,"{'neg': 0.062, 'neu': 0.892, 'pos': 0.046, 'co..."
4,Wird halt nicht jeder als Elon geboren ... a...,NaN,-0.1027,"{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'comp..."


In [58]:
print(len(df_GerVader))
print(len(df))

114444
114444


In [61]:
df_GerVader.loc[df_GerVader["text"] == " 📈 Nutze den Moment &amp; handel einfach und schnell am Kryptomarkt  "]

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed
63618,📈 Nutze den Moment &amp; handel einfach und s...,NaN,0.6124,"{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'comp..."


In [70]:
df_GerVader["tweet_id_2"] = df.loc[df_GerVader["text"] == " 📈 Nutze den Moment &amp; handel einfach und schnell am Kryptomarkt  "]["tweet_id"]

In [71]:
df_GerVader.loc[df_GerVader["text"] == " 📈 Nutze den Moment &amp; handel einfach und schnell am Kryptomarkt  "]

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed,tweet_id_2
63618,📈 Nutze den Moment &amp; handel einfach und s...,1.250657e+18,0.6124,"{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'comp...",1.250657e+18


In [69]:
df.loc[df_GerVader["text"] == " 📈 Nutze den Moment &amp; handel einfach und schnell am Kryptomarkt  "]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,sentiment_bert
63618,1250656701205884928,163537654,2020-04-16 05:26:08+00:00,1250656701205884928,de,0,0,0,0,FNTweetCenterDax30,None,None,Wirecard: Es geht abwärts,negative


In [82]:
df_GerVader.loc[df["text"] == df_GerVader.iloc[0]["text"]]["sentiment_GerVADER"]

8   -0.4019
Name: sentiment_GerVADER, dtype: float64

In [276]:
list_error_tweets = []
n = 1000
for each in range(len(df_GerVader)):
    if (each/n) == 1:
        print(each)
        print("DF_1: ", df.loc[df["text"] == df_GerVader.iloc[each]["text"]].index[0])
        print("DF_1: ", df.iloc[df.loc[df["text"] == df_GerVader.iloc[each]["text"]].index[0]]["text"])
        print("DF_2: ", df_GerVader.loc[df_GerVader["text"] == (df.loc[df["text"] == df_GerVader.iloc[each]["text"]]["text"].iloc[0])]["sentiment_GerVADER"].iloc[0])
        print("DF_2: ", df_GerVader.loc[df_GerVader["text"] == (df.loc[df["text"] == df_GerVader.iloc[each]["text"]]["text"].iloc[0])]["text"].iloc[0]) 
        n = n + 1000
    try:
        df.at[df.loc[df["text"] == df_GerVader.iloc[each]["text"]].index[0], "sentiment_vader"] = df_GerVader.loc[df_GerVader["text"] == (df.loc[df["text"] == df_GerVader.iloc[each]["text"]]["text"].iloc[0])]["sentiment_GerVADER"].iloc[0]
        
    except Exception:
        print(each)
        list_error_tweets.append(df_GerVader.iloc[each]["text"])

1000
DF_1:  4333
DF_1:     Aktie bis zum tot einfach shorten 
DF_2:  -0.4939
DF_2:     Aktie bis zum tot einfach shorten 
2000
DF_1:  7587
DF_1:  Jan Marsalek hat den Finanzdienstleister Wirecard mitgeprägt bis zu seiner fristlosen Entlassung. Seither wird er gesucht. Nun führt eine Spur nach Weißrussland. 
DF_2:  -0.1027
DF_2:  Jan Marsalek hat den Finanzdienstleister Wirecard mitgeprägt bis zu seiner fristlosen Entlassung. Seither wird er gesucht. Nun führt eine Spur nach Weißrussland. 
3000
DF_1:  14558
DF_1:  TecDax: SMA schwindet ++ Wirecard abwärts ++ Telefonica zieht an ++ Siltronic vorneweg ++ TecDax erhöht Verlust 
DF_2:  -0.6705
DF_2:  TecDax: SMA schwindet ++ Wirecard abwärts ++ Telefonica zieht an ++ Siltronic vorneweg ++ TecDax erhöht Verlust 
4000
DF_1:  11880
DF_1:  🔍 Wirecard Technologies GmbH sucht Developer Java (m/f) Division Issuing 🔎Jobs München 
DF_2:  -0.4588
DF_2:  🔍 Wirecard Technologies GmbH sucht Developer Java (m/f) Division Issuing 🔎Jobs München 
5000
DF_1:

In [280]:
list_error_tweets[0]

' Danke lieber Götz. Das frage ich mich seit gestern auch. Als Mitarbeiter im Rechnungswesen würde ich schnellstmöglich kündigen. Bin seit ca. 30 Jahren in diesem Bereich tätig.'

In [286]:
df.loc[df["text"].str.contains(list_error_tweets[0])]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,sentiment_bert,sentiment_vader
2187,1255139945586417666,1217881588882714625,2020-04-28 18:40:57+00:00,1255205376909348864,de,0,0,1,0,Twitter for Android,replied_to,1255204204278358024,Danke lieber Götz. Das frage ich mich seit ...,negative,-0.4019


In [289]:
df_GerVader.loc[df_GerVader["text"] == list_error_tweets[0]]["sentiment_GerVADER"].iloc[0]

0.6808

In [290]:
df.loc[df["text"].str.contains(list_error_tweets[0])].index[0]

2187

In [291]:
df.at[df.loc[df["text"].str.contains(list_error_tweets[0])].index[0], "sentiment_vader"] = df_GerVader.loc[df_GerVader["text"] == list_error_tweets[0]]["sentiment_GerVADER"].iloc[0]

In [293]:
df.loc[df["text"].str.contains(list_error_tweets[0])]

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,sentiment_bert,sentiment_vader
2187,1255139945586417666,1217881588882714625,2020-04-28 18:40:57+00:00,1255205376909348864,de,0,0,1,0,Twitter for Android,replied_to,1255204204278358024,Danke lieber Götz. Das frage ich mich seit ...,negative,0.6808


In [300]:
df.tail()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,sentiment_bert,sentiment_vader
114439,1164960547693105153,4584354895,2019-08-23 18:00:13+00:00,1164960547693105153,de,0,0,0,0,tickwatcher,None,None,Top Gainers (&gt;30)UVXY JNUG DRIP OPYGY URR E...,negative,0.5574
114440,1228994498610503680,704753138146877440,2020-02-17 17:53:29+00:00,1229463896425205768,de,0,0,2,0,Twitter Web App,replied_to,1229459682403725314,1/ Die Anwälte der Klägerseit...,neutral,0.0000
114441,1228017316375539712,1138485557502382082,2020-02-13 18:09:16+00:00,1228018315785838593,de,0,0,0,1,Twitter for iPhone,replied_to,1228017316375539712,Ist nicht der WRCDF LV Anteil mit zusätzlichen...,negative,0.0000
114442,1275070899822059523,1001192245,2020-06-22 14:19:26+00:00,1275070899822059523,de,0,0,0,0,TalkMarketsApp,None,None,Olaf Scholz - The German Finance Minister Is D...,neutral,0.0000
114443,1271435871364042754,392388185,2020-06-12 13:45:22+00:00,1271438445500682244,de,0,1,9,0,Twitter Web App,replied_to,1271435871364042754,"12.06. WDI 29,09 Mio. +1,16% (v...",negative,0.1027


In [299]:
df_GerVader.loc[df_GerVader["text"] == df.iloc[-1]["text"]]

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed,tweet_id_2
114443,"12.06. WDI 29,09 Mio. +1,16% (v...",NaN,0.1027,"{'neg': 0.097, 'neu': 0.795, 'pos': 0.108, 'co...",NaN


In [275]:
df.iloc[260]["text"]

'Der Zahlungsabwickler Wirecard ist in Turbulenzen. Jetzt äußert sich sein Chef, Markus Braun, zu den Vorgängen in Asien, zum Schutz durch die Bafin und zu juristischen Schritten gegen Medien. '

In [177]:
df.head()

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text,sentiment_bert,sentiment_vader
0,703311366262800384,1392131804,2016-02-26 20:11:11+00:00,703311366262800384,de,0,0,2,0,Twitter Web Client,None,None,Wirecard-Chef Markus Braun pumpt irrwitzige Su...,neutral,-0.2023
1,702405572012974080,1053723204,2016-02-24 08:11:53+00:00,702405572012974080,de,0,0,0,0,Twitter Web Client,None,None,Jetzt günstig einsammeln und später absahnen? ...,neutral,-0.2023
2,700578193334411264,19811837,2016-02-19 07:10:32+00:00,700578193334411264,de,2,0,0,0,Twitter Web Client,None,None,Wirecard: Was führt der Vorstand im Schilde? M...,neutral,-0.5971
3,700221272840601601,19811837,2016-02-18 07:32:15+00:00,700221272840601601,de,0,0,1,0,Twitter Web Client,None,None,Wirecard-Vorstand kauft weiter massiv eigene A...,negative,-0.1531
4,697324084221227008,183289696,2016-02-10 07:39:51+00:00,697324084221227008,de,0,0,0,0,dlvr.it,None,None,📈 Wirecard: Serie von Aktienkäufen des Vorstan...,neutral,-0.1027


In [182]:
df_GerVader.loc[df_GerVader["text"] == df.iloc[4]["text"]]

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed,tweet_id_2
26432,📈 Wirecard: Serie von Aktienkäufen des Vorstan...,NaN,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NaN


In [191]:
df.loc[df["text"] == df_GerVader.iloc[0]["text"]]["text"].iloc[0]

'Auf Einladung von Bürgermeister Markus Braun durfte Dr. Mathias Bellinghausen heute im alten Senatssaal des... '

In [192]:
df_GerVader.loc[df["text"] == df_GerVader.iloc[0]["text"]]["text"].iloc[0]

"Wirecard - CEO und Großaktionär Markus Braun im Gespräch mit Ursula Schwarzer zu der Frage, woher er die mehr als 60 Millionen Euro net cash für seine Aktienkäufe hat. Und was ihn zum Weinen bringt (Shortseller sind's nicht).  DAX"

In [193]:
df.loc[df["text"] == df_GerVader.iloc[0]["text"]].index

Int64Index([8], dtype='int64')

In [195]:
df_GerVader.loc[df["text"] == df_GerVader.iloc[0]["text"]]

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed,tweet_id_2
8,Wirecard - CEO und Großaktionär Markus Braun i...,NaN,-0.4019,"{'neg': 0.068, 'neu': 0.932, 'pos': 0.0, 'comp...",NaN


In [238]:
ind_num = 9

In [261]:
df.iloc[df.loc[df["text"] == df_GerVader.iloc[ind_num]["text"]].index[0]]["text"]

'Wirecard: Konzernchef Markus Braun über den zweifelhaften Ruf des Unternehmens '

In [265]:
df_GerVader.loc[df_GerVader["text"] == (df.loc[df["text"] == df_GerVader.iloc[ind_num]["text"]]["text"].iloc[0])]

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed,tweet_id_2
9,Wirecard: Konzernchef Markus Braun über den zw...,NaN,-0.3353,"{'neg': 0.209, 'neu': 0.791, 'pos': 0.0, 'comp...",NaN


In [264]:
if df.iloc[df.loc[df["text"] == df_GerVader.iloc[ind_num]["text"]].index[0]]["text"] == df_GerVader.loc[df_GerVader["text"] == (df.loc[df["text"] == df_GerVader.iloc[ind_num]["text"]]["text"].iloc[0])]["text"].iloc[0]:
    print("GEIL")

GEIL


In [167]:
df.at[df.loc[df["text"] == df_GerVader.iloc[0]["text"]].index[0], "sentiment_vader"] = -1

In [168]:
df.iloc[8]["sentiment_vader"]

-1.0

In [172]:
list_error_tweets = []
for each in range(64000,65000):
    print(each)
    try:
        df.at[df.loc[df["text"] == df_GerVader.iloc[each]["text"]].index[0], "sentiment_vader"] = df_GerVader.loc[df["text"] == df_GerVader.iloc[each]["text"]]["sentiment_GerVADER"].iloc[0]
    except Exception:
        print(each)
        list_error_tweets.append(df_GerVader.iloc[each]["text"])

64000
64001
64002
64003
64004
64005
64006
64007
64008
64009
64010
64011
64012
64013
64014
64015
64016
64017
64018
64019
64020
64021
64022
64023
64024
64025
64026
64027
64028
64029
64030
64031
64032
64033
64034
64035
64036
64037
64038
64039
64040
64041
64042
64043
64044
64045
64046
64047
64048
64049
64050
64051
64052
64053
64054
64055
64056
64057
64058
64059
64060
64061
64062
64063
64064
64065
64066
64067
64068
64069
64070
64071
64072
64073
64074
64075
64076
64077
64078
64079
64080
64081
64082
64083
64084
64085
64086
64087
64088
64089
64090
64091
64092
64093
64094
64095
64096
64097
64098
64099
64100
64101
64102
64103
64104
64105
64106
64107
64108
64109
64110
64111
64112
64113
64114
64115
64116
64117
64118
64119
64120
64121
64122
64123
64124
64125
64126
64127
64128
64129
64130
64131
64132
64133
64134
64135
64136
64137
64138
64139
64140
64141
64142
64143
64144
64145
64146
64147
64148
64149
64150
64151
64152
64153
64154
64155
64156
64157
64158
64159
64160
64161
64162
64163
64164
64165
6416

In [174]:
list_error_tweets

[' Danke lieber Götz. Das frage ich mich seit gestern auch. Als Mitarbeiter im Rechnungswesen würde ich schnellstmöglich kündigen. Bin seit ca. 30 Jahren in diesem Bereich tätig.']

In [93]:
df["sentiment_vader"].isna().sum()

114443

In [96]:
df.loc[df["sentiment_vader"].notna()]["text"]

114443     12.06. WDI              29,09 Mio.  +1,16% (v...
Name: text, dtype: object

In [97]:
df_GerVader.tail()

,text,tweet_id,sentiment_GerVADER,sentiment_GerVADER_detailed,tweet_id_2
114439,Top Decliners (&gt;30)HBANP BAC.PR.L KNMCY DJC...,NaN,0.5574,"{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'comp...",NaN
114440,HR Consulting Top Job Stellenangebote - Verk...,NaN,0.5574,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",NaN
114441,Top Gainers (&gt;30)CRWD RYAM.PR.A RRGB SKX EX...,NaN,0.5574,"{'neg': 0.0, 'neu': 0.896, 'pos': 0.104, 'comp...",NaN
114442,Top Gainers (&gt;30)UVXY JNUG DRIP OPYGY URR E...,NaN,0.5574,"{'neg': 0.0, 'neu': 0.896, 'pos': 0.104, 'comp...",NaN
114443,"12.06. WDI 29,09 Mio. +1,16% (v...",NaN,0.1027,"{'neg': 0.097, 'neu': 0.795, 'pos': 0.108, 'co...",NaN


In [301]:
#Backup CSV erstellen
df.to_csv("Dataset_DE_Bert_VADER.csv", sep=";", index=False)

In [136]:
df = pd.DataFrame([[0, 2, 3], [0, 4, 1], [10, 20, 30]],
...                   index=[4, 5, 6], columns=['A', 'B', 'C'])

In [139]:
df

,A,B,C
4,0,2,3
5,0,4,1
6,10,20,30


In [140]:
df.at[4, "B"] = 10

In [149]:
df

,A,B,C
4,0,10,3
5,0,4,1
6,10,20,30


In [158]:
df.at[(df.loc[df["A"] == 10]["A"].index[0]), "A"] = 100

In [159]:
df

,A,B,C
4,0,10,3
5,0,4,1
6,100,20,30


In [160]:
df = df_backup